<a href="https://colab.research.google.com/github/AylinNaebzadeh/Weights-and-Biases/blob/main/W%26B_101_Get_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install packages

In [33]:
!pip install wandb

In [34]:
!pip install torch>=1.9

In [35]:
!pip install torchvision

#Implementation

In [36]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [37]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [38]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [39]:
def get_predictions(model, inputs, optimizer):
    optimizer.zero_grad()
    return model(inputs)

In [40]:
def update_model(data, model, criterion, optimizer):
    inputs, labels = data
    preds = get_predictions(model, inputs, optimizer)
    loss = criterion(preds, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

In [41]:
def get_transforms(norm=0.5):
    return transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((norm, norm, norm), (norm, norm, norm))])


The dataset which is used:

The CIFAR-10 dataset consists of 60,000 32x32 colour images in 10 classes, with 6,000 images per class. It has a total of 50,000 training images and 10,000 test images which is further divided into five training batches and one test batch, each with 10,000 images.

- https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html?ref=hackernoon.com#torchvision.datasets.CIFAR10
- https://hackernoon.com/11-torchvision-datasets-for-computer-vision-you-need-to-know

In [42]:
def get_data(transforms, batch_size=4):
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transforms)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
    return trainloader

In [43]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [46]:
def train():
  # we store our configs in a dictionary.
  # More refactored, and useable for wandb
  # then we would refrence the below dictionary whenever needed
  config = {
        'norm': 0.5,
        'batch_size': 64,
        'lr': 0.002, # learning rate
        'momentum': 0.9,
        'epochs': 20
  }
  # wandb.finish()
  with wandb.init(project='GPT-5', config=config):
        config = wandb.config

        transforms = get_transforms(config.norm)
        data = get_data(transforms, config.batch_size)
        model = Net()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)

        for epoch in range(config.epochs):
            for i, batch in enumerate(data, 0):
                loss = update_model(batch, model, criterion, optimizer)

                # for training, all the prints where refactored,
                # and the wandb.log module was used insted
                # logging, and passing a dictionary of metric we try to improve
                wandb.log({'epoch': epoch, 'loss': loss})
        path = './cifar_net.pth'
        save_model(model, path)

In [47]:
train()

Files already downloaded and verified


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
loss,██▇▆▄▅▅▄▄▄▄▄▃▄▃▃▄▂▄▄▄▃▂▂▂▂▃▂▃▂▁▂▂▂▁▂▃▂▁▁
epoch,19
loss,0.68925


#TODO: Sweeps...

In [50]:
api = wandb.Api()

sweep = api.sweep("aylin-naebzadeh/GPT-5/114g96og")
runs = sorted(sweep.runs,
  key=lambda run: run.summary.get("val_acc", 0), reverse=True)
val_acc = runs[0].summary.get("val_acc", 0)
print(f"Best run {runs[0].name} with {val_acc}% validation accuracy")

runs[0].file("model.h5").download(replace=True)
print("Best model saved to model-best.h5")

IndexError: ignored